# Data Validation Report for `validation_test_case`


The validation_test_case project is a test assignment focused on verifying the accuracy of an AI-based ticket classification system.
The dataset contains tickets, which are customer reviews or opinions about a company (in this case, Booking).
The system automatically classifies each ticket into five predefined topics and determines the sentiment for each topic (positive, negative, or neutral).

The goal of this task is to validate the correctness of the system’s outputs in two key dimensions:

Topic Volume Validation — Verify whether the number of tickets assigned to each topic matches the expected or actual values.

Sentiment Validation — Assess the accuracy of sentiment classification (positive, negative, neutral) within each topic.

In the first part of report, I import data from two JSON files, validate their data, and eliminate the possibility of empty lines. In the second, I process and compare the obtained data. In the third, I display graphs demonstrating the data and draw a conclusion on the work done.

In [26]:
with open("json_file_for_validation/booking_reviews_clean.json", "r") as f:
    clean_tickets = f.read()

with open("json_file_for_validation/results_booking_reviews_clean.json", "r") as f:
    results_tickets = f.read()